# `GiRaFFE_HO` C code library: A-to-B code

### Author: Patrick Nelson

This writes and documents the C code that `GiRaFFE_HO` uses to compute the magnetic fields from the vector potential. This is a relatively straightforward calculation, but requires care to be taken in the ghost zones. 
These expressions are used to calculate the magnetic fields from the vector potential. 

We will also need to compute $B^i$ everywhere in order to evolve $\tilde{S}_i$. However, $B^i = \epsilon^{ijk} \partial_j A_k$ requires derivatives of $A_i$, so getting $B^i$ will require some finesse. A chief aspect of this will require using lower-order finite differencing in the ghost zones. To that end, we will create header files for each finite differencing order $\leq 10$, as well as upwinded- and downwinded-derivatives at 2nd order. These will let us compute the derivative at the outermost gridpoints.

In [4]:
# The A-to-B driver
from outputC import *
import grid as gri
import indexedexp as ixp
import NRPy_param_funcs as par
import finite_difference as fin

# Set spatial dimension (must be 3 for BSSN)
DIM = 3
par.set_parval_from_str("grid::DIM",DIM)

# Import the GiRaFFE_HO_v2 module and run its function to build BU
import GiRaFFE_HO.GiRaFFE_Higher_Order_v2 as gho
gho.GiRaFFE_Higher_Order_v2()

# Import the Levi-Civita symbol and build the corresponding tensor.
# We already have a handy function to define the Levi-Civita symbol in WeylScalars
import WeylScal4NRPy.WeylScalars_Cartesian as weyl
LeviCivitaDDD = weyl.define_LeviCivitaSymbol_rank3()
LeviCivitaUUU = ixp.zerorank3()
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            LCijk = LeviCivitaDDD[i][j][k]
            #LeviCivitaDDD[i][j][k] = LCijk * sp.sqrt(gho.gammadet)
            LeviCivitaUUU[i][j][k] = LCijk / sp.sqrt(gho.gammadet)

AD_dD = ixp.declarerank2("AD_dD","nosym")
BU = ixp.zerorank1() # BU is already registered as a gridfunction, but we need to zero its values and declare it in this scope.
# We can use this function to compactly reset to expressions to print at each FD order.
def set_BU_to_print():
    return [lhrh(lhs=gri.gfaccess("out_gfs","BU0"),rhs=BU[0]),\
            lhrh(lhs=gri.gfaccess("out_gfs","BU1"),rhs=BU[1]),\
            lhrh(lhs=gri.gfaccess("out_gfs","BU2"),rhs=BU[2])]            

for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

# We'll lower the FD order at each stage and write to a new file.
# But first, we need to save the original order
original_order = par.parval_from_str("finite_difference::FD_CENTDERIVS_ORDER")

par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 10)
fin.FD_outputC("GiRaFFE_standalone/B_from_A_10.h",set_BU_to_print(),params="outCverbose=False")

par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 8)
fin.FD_outputC("GiRaFFE_standalone/B_from_A_8.h",set_BU_to_print(),params="outCverbose=False")

par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 6)
fin.FD_outputC("GiRaFFE_standalone/B_from_A_6.h",set_BU_to_print(),params="outCverbose=False")

par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 4)
fin.FD_outputC("GiRaFFE_standalone/B_from_A_4.h",set_BU_to_print(),params="outCverbose=False")

par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 2)
fin.FD_outputC("GiRaFFE_standalone/B_from_A_2.h",set_BU_to_print(),params="outCverbose=False")

# For the outermost points, we'll need a separate file for each face. 
# These will correspond to an upwinded and a downwinded file for each direction.
AD_ddnD = ixp.declarerank2("AD_ddnD","nosym")
for i in range(DIM):
    BU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            if j is 0:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_ddnD[k][j]
            else:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

fin.FD_outputC("GiRaFFE_standalone/B_from_A_2x0D.h",set_BU_to_print(),params="outCverbose=False")

AD_dupD = ixp.declarerank2("AD_dupD","nosym")
for i in range(DIM):
    BU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            if j is 0:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dupD[k][j]
            else:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

fin.FD_outputC("GiRaFFE_standalone/B_from_A_2x0U.h",set_BU_to_print(),params="outCverbose=False")

for i in range(DIM):
    BU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            if j is 1:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_ddnD[k][j]
            else:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

fin.FD_outputC("GiRaFFE_standalone/B_from_A_2x1D.h",set_BU_to_print(),params="outCverbose=False")
for i in range(DIM):
    BU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            if j is 1:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dupD[k][j]
            else:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

fin.FD_outputC("GiRaFFE_standalone/B_from_A_2x1U.h",set_BU_to_print(),params="outCverbose=False")

for i in range(DIM):
    BU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            if j is 2:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_ddnD[k][j]
            else:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

fin.FD_outputC("GiRaFFE_standalone/B_from_A_2x2D.h",set_BU_to_print(),params="outCverbose=False")
for i in range(DIM):
    BU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            if j is 2:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dupD[k][j]
            else:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

fin.FD_outputC("GiRaFFE_standalone/B_from_A_2x2U.h",set_BU_to_print(),params="outCverbose=False")

# And now, we set the FD order back to what it was.
par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", original_order)

initialize_Cparam() minor warning: Did nothing; already initialized parameter smallbPoynET::M_PI
Wrote to file "GiRaFFE_standalone/B_from_A_10.h"
Wrote to file "GiRaFFE_standalone/B_from_A_8.h"
Wrote to file "GiRaFFE_standalone/B_from_A_6.h"
Wrote to file "GiRaFFE_standalone/B_from_A_4.h"
Wrote to file "GiRaFFE_standalone/B_from_A_2.h"
Wrote to file "GiRaFFE_standalone/B_from_A_2x0D.h"
Wrote to file "GiRaFFE_standalone/B_from_A_2x0U.h"
Wrote to file "GiRaFFE_standalone/B_from_A_2x1D.h"
Wrote to file "GiRaFFE_standalone/B_from_A_2x1U.h"
Wrote to file "GiRaFFE_standalone/B_from_A_2x2D.h"
Wrote to file "GiRaFFE_standalone/B_from_A_2x2U.h"


Since we need to do some extra work to fill the ghost zones when we solve for $B^i$ from $A_k$, we will put the functions in their own file to keep things cleaner.

In [ ]:
%%writefile GiRaFFE_standalone/driver_AtoB.c
#include <math.h>
#include <stdio.h>
#include <stdlib.h>

#ifndef REAL
#define REAL double
#include "NGHOSTS.h" // A NRPy+-generated file, which is set based on FD_CENTDERIVS_ORDER.
#include "../CurviBoundaryConditions/gridfunction_defines.h"
#define IDX4(g,i,j,k) \
( (i) + Nxx_plus_2NGHOSTS[0] * ( (j) + Nxx_plus_2NGHOSTS[1] * ( (k) + Nxx_plus_2NGHOSTS[2] * (g) ) ) )
#define IDX3(i,j,k) ( (i) + Nxx_plus_2NGHOSTS[0] * ( (j) + Nxx_plus_2NGHOSTS[1] * (k) ) )
// Assuming idx = IDX3(i,j,k). Much faster if idx can be reused over and over:
#define IDX4pt(g,idx)   ( (idx) + (Nxx_plus_2NGHOSTS[0]*Nxx_plus_2NGHOSTS[1]*Nxx_plus_2NGHOSTS[2]) * (g) )
#endif

// In the standalone, these are already defined by the boundary conditions module.
/*const int MAXFACE = -1;
*const int NUL     = +0;
*const int MINFACE = +1;
*/

// Declare boundary condition FACE_UPDATE function,
// which fills in the ghost zones with successively
// lower order finite differencing
void AtoB(const int ORDER, const int Nxx_plus_2NGHOSTS[3], const REAL *in_gfs, REAL *aux_gfs, const REAL dxx[3],
          const int i0min, const int i0max, 
          const int i1min, const int i1max, 
          const int i2min, const int i2max, 
          const int FACEX0, const int FACEX1, const int FACEX2) {
  
  const REAL invdx0 = 1.0 / dxx[0];
  const REAL invdx1 = 1.0 / dxx[1];
  const REAL invdx2 = 1.0 / dxx[2];

  if(ORDER==8) {
    //printf("Computing A to B with Order = 8...\n");
    for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
        #include "B_from_A_8.h"
    }
  } else if(ORDER==6) {
    //printf("Computing A to B with Order = 6...\n");
    for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
        #include "B_from_A_6.h"
    }
  } else if(ORDER==4) {
    //printf("Computing A to B with Order = 4...\n");
    for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
        #include "B_from_A_4.h"
    }
  } else if(ORDER==2) {
    //printf("Computing A to B with Order = 2...\n");
    for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
        #include "B_from_A_2.h"
    } 
  } else if(ORDER==0) {
    if(FACEX0==MAXFACE) {
    //printf("Computing A to B at x = max...\n");
        for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
            #include "B_from_A_2x0D.h"
        }
    } else if(FACEX0==MINFACE) {
    //printf("Computing A to B at x = min...\n");
        for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
            #include "B_from_A_2x0U.h"
        }
    } else if(FACEX1==MAXFACE) {
    //printf("Computing A to B at y = max...\n");
        for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
            #include "B_from_A_2x1D.h"
        }
    } else if(FACEX1==MINFACE) {
    //printf("Computing A to B at y = min...\n");
        for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
            #include "B_from_A_2x1U.h"
        }
    } else if(FACEX2==MAXFACE) {
    //printf("Computing A to B at z = max...\n");
        for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
            #include "B_from_A_2x2D.h"
        }
    } else if(FACEX2==MINFACE) {
    //printf("Computing A to B at z = min...\n");
        for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
            #include "B_from_A_2x2U.h"
        }
    } else {
        printf("ERROR. FACEX parameters not set properly.\n");
        exit(1);
    }
  } else {
    printf("ERROR. ORDER = %d not supported!\n",ORDER);
    exit(1);
  }
}

void driver_A_to_B(const int Nxx[3],const int Nxx_plus_2NGHOSTS[3], const REAL dxx[3],const REAL *in_gfs,REAL *aux_gfs) {
  
  const int *Nx = Nxx;
  int ORDER = NGHOSTS*2;
  for(int ii=0;ii<Nxx_plus_2NGHOSTS[2]*Nxx_plus_2NGHOSTS[1]*Nxx_plus_2NGHOSTS[0];ii++) {
      aux_gfs[IDX4pt(BU0GF,ii)] = 1.0 / 0.0;
      aux_gfs[IDX4pt(BU1GF,ii)] = 1.0 / 0.0;
      aux_gfs[IDX4pt(BU2GF,ii)] = 1.0 / 0.0;
  }

  //printf("Starting A to B driver with Order = %d...\n",ORDER);
  int imin[3] = { NGHOSTS, NGHOSTS, NGHOSTS };
  int imax[3] = { Nx[0]+NGHOSTS, Nx[1]+NGHOSTS, Nx[2]+NGHOSTS };
  AtoB(ORDER, Nxx_plus_2NGHOSTS, in_gfs, aux_gfs,dxx, imin[0],imax[0],imin[1],imax[1],imin[2],imax[2], NUL,NUL,NUL);
  while(ORDER>0) {
      // After updating each face, adjust imin[] and imax[] 
      //   to reflect the newly-updated face extents.
      ORDER -= 2;
      AtoB(ORDER, Nxx_plus_2NGHOSTS, in_gfs, aux_gfs,dxx, imin[0]-1,imin[0], imin[1],imax[1], imin[2],imax[2], MINFACE,NUL,NUL); 
      if(ORDER!=0) imin[0]--;
      AtoB(ORDER, Nxx_plus_2NGHOSTS, in_gfs, aux_gfs,dxx, imax[0],imax[0]+1, imin[1],imax[1], imin[2],imax[2], MAXFACE,NUL,NUL); 
      if(ORDER!=0) imax[0]++;

      AtoB(ORDER, Nxx_plus_2NGHOSTS, in_gfs, aux_gfs,dxx, imin[0],imax[0], imin[1]-1,imin[1], imin[2],imax[2], NUL,MINFACE,NUL); 
      if(ORDER!=0) imin[1]--;
      AtoB(ORDER, Nxx_plus_2NGHOSTS, in_gfs, aux_gfs,dxx, imin[0],imax[0], imax[1],imax[1]+1, imin[2],imax[2], NUL,MAXFACE,NUL); 
      if(ORDER!=0) imax[1]++;

      AtoB(ORDER, Nxx_plus_2NGHOSTS, in_gfs, aux_gfs,dxx, imin[0],imax[0], imin[1],imax[1], imin[2]-1,imin[2], NUL,NUL,MINFACE); 
      if(ORDER!=0) imin[2]--;
      AtoB(ORDER, Nxx_plus_2NGHOSTS, in_gfs, aux_gfs,dxx, imin[0],imax[0], imin[1],imax[1], imax[2],imax[2]+1, NUL,NUL,MAXFACE); 
      if(ORDER!=0) imax[2]++;
    }
}
